# Jupyter notebook to compute metrics for performance comparison
## importing packages and building dataset path

In [1]:
from pathlib import Path
import lpips
import numpy
import glob
import torch
from skimage import metrics
from torchvision import transforms
from pytorch_msssim import ms_ssim
from PIL import Image
from datetime import datetime
from numpy import mean

device = 'cuda' if torch.cuda.is_available() else 'cpu'

loss_fn_alex = lpips.LPIPS(net='alex')

current_path = Path('.').resolve()
print(str(current_path))

reference_path = str(current_path)+'/IMAGES/PNG_IMAGES'
print("Reference source: ",reference_path)

/home/filippo/anaconda3/envs/imageCompressionAI/lib/python3.11/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'libc10_cuda.so: cannot open shared object file: No such file or directory'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]


/home/filippo/anaconda3/envs/imageCompressionAI/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/filippo/anaconda3/envs/imageCompressionAI/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Loading model from: /home/filippo/anaconda3/envs/imageCompressionAI/lib/python3.11/site-packages/lpips/weights/v0.1/alex.pth
/home/filippo/DataspellProjects/ImageCompressionAI
Reference source:  /home/filippo/DataspellProjects/ImageCompressionAI/IMAGES/PNG_IMAGES


## Preparing vector of folders to compare

In [2]:
datasets = {
    'JPEG_1':'/IMAGES/JPEG/2',
    'JPEG_2':'/IMAGES/JPEG/6',
    'JPEG_3':'/IMAGES/JPEG/19',
    'JPEG_4':'/IMAGES/JPEG/23',
    'JPEG_5':'/IMAGES/JPEG/30',
    'JPEG2000_1':'/IMAGES/JPEG2000/175',
    'JPEG2000_2':'/IMAGES/JPEG2000/140',
    'JPEG2000_3':'/IMAGES/JPEG2000/115',
    'JPEG2000_4':'/IMAGES/JPEG2000/72',
    'JPEG2000_5':'/IMAGES/JPEG2000/56',
    'BPG_1':'/IMAGES/BPG/49',
    'BPG_2':'/IMAGES/BPG/44',
    'BPG_3':'/IMAGES/BPG/41',
    'BPG_4':'/IMAGES/BPG/38',
    'BPG_5':'/IMAGES/BPG/36',
    'VVC_1':'/IMAGES/VVC/42',
    'VVC_2':'/IMAGES/VVC/36',
    'VVC_3':'/IMAGES/VVC/33',
    'VVC_4':'/IMAGES/VVC/31',
    'VVC_5':'/IMAGES/VVC/30',
    'mbt2018_1':'/IMAGES/NET/mbt2018_1',
    'mbt2018_2':'/IMAGES/NET/mbt2018_2',
    'mbt2018_3':'/IMAGES/NET/mbt2018_3',
    'mbt2018_4':'/IMAGES/NET/mbt2018_4',
    'mbt2018_5':'/IMAGES/NET/mbt2018_5',
    'mbt2018_mean_1':'/IMAGES/NET/mbt2018_mean_1',
    'mbt2018_mean_2':'/IMAGES/NET/mbt2018_mean_2',
    'mbt2018_mean_3':'/IMAGES/NET/mbt2018_mean_3',
    'mbt2018_mean_4':'/IMAGES/NET/mbt2018_mean_4',
    'mbt2018_mean_5':'/IMAGES/NET/mbt2018_mean_5',
    'cheng2020_attn_1':'/IMAGES/NET/cheng2020_attn_1',
    'cheng2020_attn_2':'/IMAGES/NET/cheng2020_attn_2',
    'cheng2020_attn_3':'/IMAGES/NET/cheng2020_attn_3',
    'cheng2020_attn_4':'/IMAGES/NET/cheng2020_attn_4',
    'cheng2020_attn_5':'/IMAGES/NET/cheng2020_attn_5',
    'cheng2020-anchor_1':'/IMAGES/NET/cheng2020-anchor_1',
    'cheng2020-anchor_2':'/IMAGES/NET/cheng2020-anchor_2',
    'cheng2020-anchor_3':'/IMAGES/NET/cheng2020-anchor_3',
    'cheng2020-anchor_4':'/IMAGES/NET/cheng2020-anchor_4',
    'cheng2020-anchor_5':'/IMAGES/NET/cheng2020-anchor_5' 
}

## Computing metrics

In [3]:
for name, path in datasets.items():
    psnr_list=[]
    mssim_list=[]
    lpips_list=[]
    count = 0
    compressed_path = str(current_path)+path
    log = open(str(compressed_path)+'/metrics.txt', 'w')
    log.writelines(['Evaluating '+name+' with dataset: '+str(compressed_path) + ' on date:' + str(datetime.now()) + '\n'])
    print('Evaluating '+name+' with dataset: '+str(compressed_path))
    for file in glob.glob(str(compressed_path)+'/*.png'):
        reference_image = Image.open(reference_path+'/'+Path(file).stem+'.png')
        reference = transforms.ToTensor()(reference_image).unsqueeze(0).to(device)
        compressed_image = Image.open(file)
        compressed = transforms.ToTensor()(compressed_image).unsqueeze(0).to(device)
        psnr = metrics.peak_signal_noise_ratio(numpy.array(reference_image),numpy.array(compressed_image),data_range=255)
        psnr_list.append(psnr)
        reference_image.close()
        compressed_image.close()
        mssim = ms_ssim(reference, compressed, data_range=1, size_average=False)
        mssim_list.append(mssim.item())
        d = loss_fn_alex(transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))(reference),transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))(compressed))
        lpips_list.append(d[0].item())
        log_string = str(Path(file).stem) + '\tPSNR: ' + str(psnr) + ' dB' + '\tMS-SSIM: ' + str(mssim.item()) + '\tLPIPS_AlexNet: ' + str(d[0].item()) + '\n'
        log.writelines([log_string])
        print(log_string)
        count += 1
        
    log.writelines(['\n',str('Average in ' + str(count) + ' images\nPSNR: ' + str(mean(psnr_list)) + 'dB\tMS-SSIM: ' + str(mean(mssim_list)) + '\tLPIPS_AlexNet: ' + str(mean(lpips_list)))])
    print(str('Average in ' + str(count) + ' images\nPSNR: ' + str(mean(psnr_list)) + 'dB\tMS-SSIM: ' + str(mean(mssim_list)) + '\tLPIPS_AlexNet: ' + str(mean(lpips_list))))
    log.close()

Evaluating VVC_1 with dataset: /home/filippo/DataspellProjects/ImageCompressionAI/IMAGES/VVC/42
IMG0001	PSNR: 28.41212904349429 dB	MS-SSIM: 0.9405682682991028	LPIPS_AlexNet: 0.26251399517059326
IMG0002	PSNR: 27.106729288623804 dB	MS-SSIM: 0.8732433915138245	LPIPS_AlexNet: 0.30482017993927
IMG0003	PSNR: 30.314984649285137 dB	MS-SSIM: 0.9633026123046875	LPIPS_AlexNet: 0.29058972001075745
IMG0004	PSNR: 28.75642282498227 dB	MS-SSIM: 0.930821418762207	LPIPS_AlexNet: 0.27686867117881775
IMG0005	PSNR: 27.761261352906182 dB	MS-SSIM: 0.9579716324806213	LPIPS_AlexNet: 0.17940738797187805
IMG0006	PSNR: 28.10852391957627 dB	MS-SSIM: 0.9509099125862122	LPIPS_AlexNet: 0.2156308889389038
IMG0007	PSNR: 30.393622347275063 dB	MS-SSIM: 0.9649496078491211	LPIPS_AlexNet: 0.292732834815979
IMG0008	PSNR: 26.32019545869594 dB	MS-SSIM: 0.9246034622192383	LPIPS_AlexNet: 0.37946832180023193
IMG0009	PSNR: 29.22943253920112 dB	MS-SSIM: 0.9334524273872375	LPIPS_AlexNet: 0.5371919274330139
IMG0010	PSNR: 29.904517368